## Imports

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"

import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


batch_size = 512
num_classes = 10
img_rows, img_cols = 28, 28

Using TensorFlow backend.


## Load data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Create data for network

In [3]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Create f1 score

In [4]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Generate model

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', f1])

## Create callbacks

In [6]:
early_stopping = EarlyStopping(monitor='val_acc', mode='max', patience=5, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.5, patience=3, verbose=1)
callbacks=[early_stopping, model_checkpoint, reduce_lr]

## Fit model

In [7]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10000,
          callbacks=callbacks,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10000
60000/60000 [==============================] - 10s 172us/step - loss: 0.5248 - acc: 0.8366 - f1: 0.8073 - val_loss: 0.1151 - val_acc: 0.9671 - val_f1: 0.9677

Epoch 00001: val_acc improved from -inf to 0.96710, saving model to best_model.h5
Epoch 2/10000
60000/60000 [==============================] - 2s 40us/step - loss: 0.1453 - acc: 0.9573 - f1: 0.9572 - val_loss: 0.0627 - val_acc: 0.9804 - val_f1: 0.9801

Epoch 00002: val_acc improved from 0.96710 to 0.98040, saving model to best_model.h5
Epoch 3/10000
60000/60000 [==============================] - 3s 42us/step - loss: 0.1022 - acc: 0.9699 - f1: 0.9705 - val_loss: 0.0499 - val_acc: 0.9843 - val_f1: 0.9846

Epoch 00003: val_acc improved from 0.98040 to 0.98430, saving model to best_model.h5
Epoch 4/10000
60000/60000 [==============================] - 2s 41us/step - loss: 0.0793 - acc: 0.9763 - f1: 0.9766 - val_loss: 0.0425 - val_acc: 0.9864 - val_f1: 0.9868

Epoch 00004:

## Score model

In [8]:
model = load_model('best_model.h5', custom_objects={'f1': f1})
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test f1_score:', score[2])

Test loss: 0.024616236625222974
Test accuracy: 0.9925
Test f1_score: 0.9924348064422608
